In [1]:
! pip install flair

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/388.3 kB ? eta -:--:--
     - -------------------------------------- 10.2/388.3 kB ? eta -:--:--
     -- ---------------------------------- 30.7/388.3 kB 435.7 kB/s eta 0:00:01
     --- --------------------------------- 41.0/388.3 kB 393.8 kB/s eta 0:00:01
     --- --------------------------------- 41.0/388.3 kB 393.8 kB/s eta 0:00:01
     ----- ------------------------------- 61.4/388.3 kB 297.7 kB/s eta 0:00:02
     ----- ------------------------------- 61.4/388.3 kB 297.7 kB/s eta 0:00:02
     ------- ----------------------------- 81.9/388.3 kB 286.7 kB/s eta 0:00:02
     ------- ----------------------------- 81.9/388.3 kB 286.7 kB/s eta 0:00:02
     ------- ----------------------------- 81.9/388.3 kB 286.7 kB/s eta 0:00:02
     ------- ----------------------------- 81.9/388.3 kB 286.7 kB/s eta 0:00:02
     ------- ----------------------------- 81.9/388.3 k

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.19.0 requires protobuf<4,>=3.12, but you have protobuf 4.23.4 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Flair uses PyTorch/TensorFlow in under the hood, so it's essential that you also have one of the two libraries (or both) installed.

In [2]:
import flair
#english language model for sentiment analysis
model = flair.models.TextClassifier.load('en-sentiment')

2024-06-18 13:09:23,638 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to C:\Users\matts\AppData\Local\Temp\tmpzx5zcgqf


100%|██████████| 253M/253M [31:32<00:00, 140kB/s]    

2024-06-18 13:40:56,813 copying C:\Users\matts\AppData\Local\Temp\tmpzx5zcgqf to cache at C:\Users\matts\.flair\models\sentiment-en-mix-distillbert_4.pt


2024-06-18 13:40:57,054 removing temp file C:\Users\matts\AppData\Local\Temp\tmpzx5zcgqf


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Our next step is to tokenize input text. For this we use the Flair Sentence object, which we initialize by passing our text into it:

In [4]:
text = "I like you. I love you"  # we are expecting a confidently positive sentiment here

sentence = flair.data.Sentence(text)

sentence

Sentence[7]: "I like you. I love you"

In [5]:
model.predict(sentence)
# The predict method doesn't output our prediction, instead the predictions are added to our sentence:

sentence

Sentence[7]: "I like you. I love you" → POSITIVE (0.9933)

In [6]:
sentence.get_labels()

['Sentence[7]: "I like you. I love you"'/'POSITIVE' (0.9933)]

In [7]:
sentence.get_labels()[0].value

'POSITIVE'

Now let's try with `nft` related posts extracted from the crawler


In [8]:
post = '📊  Fintech Focus 🔎  📢  NFTs - Are Non-Fungible Tokens The Next Big Thing  ⌛️ or Just Hype? Read my latest post  👇 to learn more and share your thoughts/experiences/ideas! #fintech #friday #nfts #nftcommunity #blockchain #cryptocurrency #cryptoexchange #art'
sentence = flair.data.Sentence(post)

sentence

Sentence[52]: "📊  Fintech Focus 🔎  📢  NFTs - Are Non-Fungible Tokens The Next Big Thing  ⌛ or Just Hype? Read my latest post  👇 to learn more and share your thoughts/experiences/ideas! #fintech #friday #nfts #nftcommunity #blockchain #cryptocurrency #cryptoexchange #art"

In [9]:
model.predict(sentence)
sentence.get_labels()

['Sentence[52]: "📊  Fintech Focus 🔎  📢  NFTs - Are Non-Fungible Tokens The Next Big Thing  ⌛ or Just Hype? Read my latest post  👇 to learn more and share your thoughts/experiences/ideas! #fintech #friday #nfts #nftcommunity #blockchain #cryptocurrency #cryptoexchange #art"'/'POSITIVE' (0.8608)]

This post content could be more considered as negative than positive since it is questioning the NFTs hype, or even neutral but not at all positive. So we can conclude the model is probably not going to work just like this. We should built our own or try to adjust it.

I will experiment to see if taking out the hasthags improve the true content

In [10]:
post = '📊  Fintech Focus 🔎  📢  NFTs - Are Non-Fungible Tokens The Next Big Thing  ⌛️ or Just Hype? Read my latest post  👇 to learn more and share your thoughts/experiences/ideas! #fintech #friday #nfts #nftcommunity #blockchain #cryptocurrency #cryptoexchange #art'
sentence = flair.data.Sentence(post)
model.predict(sentence)
sentence.get_labels()

['Sentence[52]: "📊  Fintech Focus 🔎  📢  NFTs - Are Non-Fungible Tokens The Next Big Thing  ⌛ or Just Hype? Read my latest post  👇 to learn more and share your thoughts/experiences/ideas! #fintech #friday #nfts #nftcommunity #blockchain #cryptocurrency #cryptoexchange #art"'/'POSITIVE' (0.8608)]

In [11]:
import re

def remove_hashtags(text):
    # Use a regular expression to find and remove hashtags
    #replaces every hashtag with a ''
    cleaned_text = re.sub(r'#\w+', '', text)
    return cleaned_text.strip()


# Removing hashtags

I did this function for removing hashtags since I noticed that they are sometimes like "spam words" which are not necessarily helpful. 

Example:

 *'📊  Fintech Focus 🔎  📢  NFTs - Are Non-Fungible Tokens The Next Big Thing  ⌛️ or Just Hype? Read my latest post  👇 to learn more and share your thoughts/experiences/ideas! #fintech #friday #nfts #nftcommunity #blockchain #cryptocurrency #cryptoexchange #art'* 

 If I remove the hashtags, it has a negative score. With hashtags it has a positive score.

 In this case. For example: #art #friday #fintech  are just noise, since they are not actually related with the title.

In [ ]:
post = '📊  Fintech Focus 🔎  📢  NFTs - Are Non-Fungible Tokens The Next Big Thing  ⌛️ or Just Hype? Read my latest post  👇 to learn more and share your thoughts/experiences/ideas! #fintech #friday #nfts #nftcommunity #blockchain #cryptocurrency #cryptoexchange #art'
#
post = remove_hashtags(post)
sentence = flair.data.Sentence(post)
model.predict(sentence)
sentence.get_labels()

['Sentence[36]: "📊  Fintech Focus 🔎  📢  NFTs - Are Non-Fungible Tokens The Next Big Thing  ⌛ or Just Hype? Read my latest post  👇 to learn more and share your thoughts/experiences/ideas!"'/'NEGATIVE' (0.9315)]

In [13]:
sentence.get_labels()[0].value

'NEGATIVE'

We can see that removing the hashtags definitely improved the prediction. So now, let's test it with more post contents.

In [14]:
import os
def txt_to_list(file_path):

    lines_list = []

    try:
        with open(file_path, 'r') as file:
            for line in file:
                # Append each line as a string to the list (remove newline characters)
                lines_list.append(line.strip())
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

    return lines_list


In [15]:
path = 'descriptions.txt'

posts = txt_to_list(path)
posts[:2]

["Last week I had the pleasure to NFT Paris by the hand of Arianee . Witnessing the transformative power of Web3 and NFTs in reshaping the concept of ownership, particularly in relation to our personal data, left me inspired and eager to delve deeper into the evolving landscape of decentralized technologies. The event sparked valuable insights, and I'm excited to continue navigating the dynamic realm of blockchain innovations. #NFTParis #Web3 #OwnershipRevolution",
 'Hello everyone Today, my friends Asma Ghamacha , Hermes Yan NTJAM NDJENG , Harold Geumtcheng , Aloys Aymrick Nzooh , Bryan Fozame and I had the chance to be part of the NFT Paris conference thanks to our school aivancity School for Technology, Business & Society Paris-Cachan where we learned a wealth of new information about blockchain, metaverse, web3 and its use cases across various industries such as finance, gaming, luxury, and more. During this enI had the privilege to engage in discussions with numerous brilliant web

In [16]:
for i in range(len(posts)):
    posts[i] = remove_hashtags(posts[i])
    if posts[i] == '':
        posts[i] = 'This is a neutral comment'

posts



["Last week I had the pleasure to NFT Paris by the hand of Arianee . Witnessing the transformative power of Web3 and NFTs in reshaping the concept of ownership, particularly in relation to our personal data, left me inspired and eager to delve deeper into the evolving landscape of decentralized technologies. The event sparked valuable insights, and I'm excited to continue navigating the dynamic realm of blockchain innovations.",
 'Hello everyone Today, my friends Asma Ghamacha , Hermes Yan NTJAM NDJENG , Harold Geumtcheng , Aloys Aymrick Nzooh , Bryan Fozame and I had the chance to be part of the NFT Paris conference thanks to our school aivancity School for Technology, Business & Society Paris-Cachan where we learned a wealth of new information about blockchain, metaverse, web3 and its use cases across various industries such as finance, gaming, luxury, and more. During this enI had the privilege to engage in discussions with numerous brilliant web3 developers and CEOs from companies 

In [28]:
import pandas as pd

df = pd.DataFrame(columns=['content', 'score', 'prediction'])

for i in range(len(posts)):
    sentence = flair.data.Sentence(posts[i])
    model.predict(sentence)
    
    # Extract relevant information from sentence and append to the DataFrame
    content = posts[i]
    score = sentence.labels[0].score
    prediction = sentence.labels[0].value
    
    print('content: ', content[:15], 'score: ', score, 'prediction: ', prediction)


content:  Last week I had score:  0.9998553991317749 prediction:  POSITIVE
content:  Hello everyone  score:  0.9989281296730042 prediction:  POSITIVE
content:  The digital wor score:  0.998602569103241 prediction:  POSITIVE
content:  Can’t wait to w score:  0.999728262424469 prediction:  POSITIVE
content:  DualMint is set score:  0.9997445940971375 prediction:  POSITIVE
content:  This Week on RW score:  0.9849713444709778 prediction:  POSITIVE
content:  Join me Saturda score:  0.973907470703125 prediction:  POSITIVE
content:  NFT Paris 2024  score:  0.9457706212997437 prediction:  POSITIVE
content:  Hello everyone  score:  0.9994600415229797 prediction:  POSITIVE
content:  “Comment achete score:  0.907588005065918 prediction:  POSITIVE
content:  A guide to   an score:  0.9981526732444763 prediction:  POSITIVE
content:  For a long time score:  0.9995279312133789 prediction:  POSITIVE
content:  Brands can now  score:  0.9996111989021301 prediction:  POSITIVE
content:  neutral score:  0.9

# Next steps for Arianee
 - Get more data through my crawler - basically iterating through more pages
 - Label the data myself as positive, negative, or neutral
 - Train a new transformer model with more data

https://towardsdatascience.com/text-classification-with-state-of-the-art-nlp-library-flair-b541d7add21f

In [1]:
#from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer

/Users/matiasmendez/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#corpus = NLPTaskDataFetcher.load_classification_corpus(Path('./'), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')
word_embeddings = [WordEmbeddings('glove'), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]
document_embeddings = DocumentLSTMEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
#substitute for my corpus using NLPTaskDataFetcher
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)